In [6]:
import networkx as nx
from graphrole import RecursiveFeatureExtractor, RoleExtractor
import torch.nn.functional as F
import torch
from model.layers import GraphSpectralFilterLayer, AnalysisFilter
from model.spectral_filter import Graph
from random import seed as rseed
from numpy.random import seed as nseed
import time
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_random_state

rseed(729)
nseed(729)
torch.manual_seed(729)


In [24]:
from sklearn.datasets import load_iris
iris= load_iris()
# Store features matrix in X
x= torch.tensor(iris.data).float()
#Store target vector in
y= torch.tensor(iris.target)

In [26]:
from sklearn.model_selection import train_test_split

train_idx, test_idx, _, _ = train_test_split(list(range(x.shape[0])), list(range(x.shape[0])), test_size=0.6, random_state=42)
val_idx, test_idx, _, _ = train_test_split(test_idx, test_idx, test_size=0.5)


def index_to_mask(index, size):
    mask = torch.zeros(size, dtype=torch.bool, device=index.device)
    mask[index] = 1
    return mask

train_mask = index_to_mask(torch.tensor(train_idx), y.shape[0])
val_mask = index_to_mask(torch.tensor(val_idx), y.shape[0])
test_mask = index_to_mask(torch.tensor(test_idx), y.shape[0])
print(train_mask.nonzero().shape)

torch.Size([60, 1])


In [27]:
G = nx.complete_graph(x.shape[0])

In [34]:
dropout = 0.
heads = 1
N = G.number_of_nodes()
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        adj = torch.sparse_coo_tensor(torch.tensor(list(G.edges)).T, torch.ones(G.number_of_edges()), (N, N))
        self.G = Graph(adj)
        self.G.estimate_lmax()

        self.analysis = GraphSpectralFilterLayer(self.G, x.shape[1], 64,
                                                 dropout=dropout, out_channels=heads, alpha=0.2, chebyshev_order=16, concat=False)

        self.synthesis = GraphSpectralFilterLayer(self.G, 64 * heads, y.max()+1, dropout=dropout,
                                                  out_channels=1, alpha=0.2,
                                                  chebyshev_order=16, concat=False)


    def reset_parameters(self):
        self.analysis.reset_parameters()
        self.synthesis.reset_parameters()

    def forward(self, input):
        x = self.analysis(input)[0]
        x = F.dropout(x, p=dropout, training=self.training)
        x = self.synthesis(x)[0]
        x = F.elu(x)

        return F.log_softmax(x, dim=1), None, None


In [35]:
from torch.optim import Adam
from sklearn.metrics import f1_score
import numpy as np

model = Net()

model.reset_parameters()
optimizer = Adam(model.parameters(), lr=0.001)

best_val_loss = float('inf')
best_val_acc = float(0)
eval_info_early_model = None
bad_counter = 0

masks = {
    'train_mask': train_mask,
    'val_mask': val_mask,
    'test_mask': test_mask,
}

for epoch in range(1, 1000):
    model.train()
    optimizer.zero_grad()
    out = model(x)[0]
    loss = F.nll_loss(out[train_mask], y[train_mask])
    loss.backward()
    optimizer.step()

    model.eval()

    with torch.no_grad():
        logits = model(x)[0]

    outs = {}
    for key in ['train', 'val', 'test']:
        mask = masks['{}_mask'.format(key)]
        loss = F.nll_loss(logits[mask], y[mask]).item()

        outs['{}_loss'.format(key)] = loss
        outs['{}_acc'.format(key)] = f1_score(y[mask].cpu(), logits[mask].max(1)[1].cpu(), average='micro')
        outs['{}_macro_f1'.format(key)] = f1_score(y[mask].cpu(), logits[mask].max(1)[1].cpu(), average='macro')

    outs['epoch'] = epoch
    if epoch % 10 == 0:
        print(outs)

    if outs['val_acc'] > best_val_acc or outs['val_loss'] < best_val_loss:
        if outs['val_acc'] >= best_val_acc:
            eval_info_early_model = outs
            torch.save(model.state_dict(), './best_barbell.pkl')
        best_val_acc = np.max((best_val_acc, outs['val_acc']))
        best_val_loss = np.min((best_val_loss, outs['val_loss']))
        bad_counter = 0
    else:
        bad_counter += 1
        if bad_counter == 10:
            break
print(eval_info_early_model)

model.load_state_dict(torch.load('./best_barbell.pkl'))
model.eval()

{'train_loss': 1.0720845460891724, 'train_acc': 0.4000000000000001, 'train_macro_f1': 0.1904761904761905, 'val_loss': 1.1219440698623657, 'val_acc': 0.35555555555555557, 'val_macro_f1': 0.17486338797814208, 'test_loss': 1.1552826166152954, 'test_acc': 0.2222222222222222, 'test_macro_f1': 0.1212121212121212, 'epoch': 10}
{'train_loss': 1.0653417110443115, 'train_acc': 0.4000000000000001, 'train_macro_f1': 0.1904761904761905, 'val_loss': 1.1141564846038818, 'val_acc': 0.37777777777777777, 'val_macro_f1': 0.2128654970760234, 'test_loss': 1.1269581317901611, 'test_acc': 0.2222222222222222, 'test_macro_f1': 0.1212121212121212, 'epoch': 20}
{'train_loss': 1.0587294101715088, 'train_acc': 0.4000000000000001, 'train_macro_f1': 0.1904761904761905, 'val_loss': 1.102262258529663, 'val_acc': 0.37777777777777777, 'val_macro_f1': 0.2128654970760234, 'test_loss': 1.1228251457214355, 'test_acc': 0.2222222222222222, 'test_macro_f1': 0.1212121212121212, 'epoch': 30}
{'train_loss': 1.051590085029602, 'tr

Net(
  (analysis): GraphSpectralFilterLayer (4 -> 64)
  (synthesis): GraphSpectralFilterLayer (64 -> tensor(3))
)